In [22]:
import pandas as pd
data = pd.read_csv("./data/processed/final.csv")
data.head()

,question,answer,question_clean,answer_clean
0,Chỉ số xét nghiệm ca 125 là gì?,Xét nghiệm CA 125 chính là chỉ số kháng nguyên...,chỉ_số xét_nghiệm ca 125,xét_nghiệm ca 125 chỉ_số kháng_nguyên ung_thư ...
1,3 món ăn đơn giản giúp ngực to một cách tự nhiên,Có thể bạn không biết những món ăn đơn giản th...,3 món đơn_giản giúp ngực to tự_nhiên,món đơn_giản thường_ngày công_dụng ngực to căn...
2,Giá khám tổng quát định kì,Như các bạn biết thì chi phí khám sức khỏe tổn...,giá khám tổng_quát định_kì,chi_phí khám sức_khỏe tổng_quát khoản khám sức...
3,Đừng để bị giảm thị lực vĩnh viễn do viêm bờ mi,"Bạn có thể gặp nhiều phiền toái, mất thẩm mĩ v...",đừng thị_lực vĩnh_viễn viêm bờ mi,phiền_toái thẩm_mĩ mắt viêm bờ mi có_thể bị th...
4,5 thời điểm vàng bạn nên uống mật ong,Bạn dùng mật ong vào buổi sáng để lọc sạch cặn...,5 vàng uống mật_ong,mật_ong lọc sạch cặn bẩn đêm detox detox mật_o...


In [23]:
import multiprocessing
num_cores = multiprocessing.cpu_count() # số core cpu máy tính

In [24]:
num_cores

8

In [25]:
# Mô hình gensim Doc2Vec bao gồm vector word cũng như vector document
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

In [26]:
# simple_preprocess là xử lý bỏ qua ký hiệu và dấu chấm câu ...
from gensim.utils import simple_preprocess

In [27]:
training_corpus = []
for i, text in enumerate(data['question_clean']):
    tagged_doc = TaggedDocument(text, [i])
    training_corpus.append(tagged_doc)

In [28]:
model = Doc2Vec(size=300, min_count=2, workers=num_cores, iter=1000)

/home/thang/env/lib/python3.6/site-packages/gensim/models/doc2vec.py:315: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/home/thang/env/lib/python3.6/site-packages/gensim/models/doc2vec.py:319: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [29]:
model.build_vocab(training_corpus)

In [30]:
model.train(training_corpus, total_examples=model.corpus_count, epochs=model.iter)

/home/thang/env/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


In [31]:
len(model.docvecs)

9148

In [32]:
from src.cleaner import Cleaner
clean = Cleaner()

In [33]:
import numpy as np
def consine(x,y):
    x = np.array(x)
    y = np.array(y)
    return x.dot(y)/(np.linalg.norm(x)*np.linalg.norm(y))

In [34]:
def get_similary(text, n):
    """ Select n question that similary with text """
    text = clean.run(text)
    input_embed = model.infer_vector([text], steps=1000)
    result = {}
    for i, q in enumerate(data['question']):
        result[q] = consine(input_embed, model.docvecs[i])
    return sorted(result.items(), key=lambda x: x[1], reverse=True)[:n]

In [35]:
%%time
get_similary("bệnh viêm gan b là gì?", 5)

CPU times: user 175 ms, sys: 0 ns, total: 175 ms
Wall time: 174 ms


[('Nhiễm khuẩn đường tiết niệu rất dễ xảy ra khi mang thai', 0.20854746),
 ('Phòng ngừa và điều trị tiểu đường thai kỳ', 0.20776266),
 ('Tại sao mọi người vẫn thường xuyên đi làm khi bị ốm', 0.20190729),
 ('Niềng răng móm có đau nhiều không?', 0.196333),
 ('Thường xuyên bị nổi mề đay buổi tối có nguy hiểm không?', 0.1894921)]

In [36]:
%%time
get_similary("làm gì khi bị ung thư?", 5)

CPU times: user 163 ms, sys: 0 ns, total: 163 ms
Wall time: 163 ms


[('Tại sao không nên trì hoãn lịch tiêm chủng', 0.15889555),
 ('Bao lâu thì nên lấy cao răng một lần?', 0.15339643),
 ('Vì sao thị lực của mẹ bầu giảm?', 0.15177925),
 ('Những lợi ích về sức khỏe của gạo lứt', 0.14693294),
 ('Các loại bao cao su chất lượng nên dùng', 0.14648248)]

In [37]:
%%time
get_similary("tôi cảm thấy mình bị HIV? Tôi phải làm gì?", 5)

CPU times: user 178 ms, sys: 5.23 ms, total: 183 ms
Wall time: 178 ms


[('Chia sẻ kinh nghiệm cắt trĩ bằng phương pháp Longo', 0.16473188),
 ('Những thức ăn bổ sung kẽm tốt cho cơ thể', 0.15822855),
 ('12 loại thực phẩm bà bầu ăn giúp \\', 0.1530132),
 ('Giải đáp thắc mắc: Dùng lăn khử mùi có gây ung thư vú không?', 0.15151656),
 ('Những biến chứng của trĩ hậu sản các mẹ cần biết', 0.14782003)]